In [1]:
!pip install flask_ngrok

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
!pip install sklearn

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pickle
import requests

In [ ]:
import os
from flask import Flask, request,jsonify
import pandas as pd
import requests
from flask_ngrok import run_with_ngrok
import sklearn

import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pickle
import requests


app = Flask(__name__)
# run_with_ngrok(app)
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="hackwestern-296303-1a10456e3ecc.json"
params = {
    'access_key': '3789340d5263ca80c40bd062e28e8bad',
    'query': 'Dubai'
}
@app.route('/')
def home():
    return "hello!"
@app.route('/weather',methods=['GET'])
def weather():
    api_result = requests.get('http://api.weatherstack.com/current', params)

    api_response = api_result.json()

    place = api_response['location']['name']
    temp = api_response['current']['temperature']
    precip_rain = api_response['current']['precip']
    precip_snow = 0
    wind = api_response['current']['wind_speed']
    weather = "sunny"

    # weather is given in a weird code
    weatherCode = api_response['current']['weather_code']
    # less than 113 is sunny
    if (weatherCode <= 113):
        weather = "sunny"
    # between 113 and 143 is cloudy
    elif (weatherCode <= 143):
        weather = "cloudy"
    # between 122 and 230 is snowing
    elif (weatherCode < 230):
        weather = "snow"
    # between 230 and 326
    elif (weatherCode < 326):
        weather = "rain"
    # everything above 326 is snow again
    else:
        weather = "snow"
    # set the precipitation to either snow or rain
    # default set to rain, but if its snowing then swap
    if (weather == "snow"):
        precip_snow = precip_rain
        precip_rain = 0
    import csv
    csv_columns = ['Wind Speed','average temperature','precipitation','snow fall']
    dict_data = [
        {'Wind Speed':wind, 'average temperature': temp, 'precipitation': precip_rain,'snow fall':25}
    ]
    csv_file = "Names.csv"
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in dict_data:
                writer.writerow(data)
    except IOError:
        print("I/O error")
    data = pd.read_csv("Names.csv")
    model= pickle.load(open('model.sav', 'rb'))
    output = model.predict(data)
    out= output.tolist()
    Temp_item = [["Shorts", "Tanktops", "T-shirts"], ["Shorts", "Tanktops", "T-shirts"],
                 ["Jeans", "T-shirts", "Sweatpants", "Full sleeved shirts"],
                 ["Hoodies", "Sweaters", "Light jackets"], ["Hoodies", "Sweaters", "Light jackets"],
                 ["Light jackets", "Wind breakers"],
                 ["Coats", "Jackets"], ["Winter jackets", "Toques"], ["Winter jackets", "Toques", "Scarfs"],
                 ["Winter jackets", "Toques", "Scarfs", "Gloves"]]
    Rain_item = [[""],["Umbrella"], ["Umbrella", "Raincoat", "Waterproof boots/shoes"], ["Snow boots"],
                 ["Snow boots", "Snow pants", "Earmufs"]]
    wet =int((out[0]-out[0]%10)/10)
    warmth = (out[0]%10)
    rain = "there is no rain, "
    heat ="it is freezing outside "
    if warmth <2:
        heat= "It is very hot outside "
    elif warmth <4:
        heat = "It is fairly warm outside "
    elif warmth <6:
        heat ="It is fairly moderate outside "
    elif warmth <8:
        heat = "It is cold outside "
    if wet ==1:
        rain= "there is light rain, "
    elif wet ==2:
        rain= "there is heavy rain, "
    elif wet ==3:
        rain= "there is light snow, "
    elif wet ==4:
        rain= "there is heavy snow, "
#     sending = heat + "and " + rain + "here are some recomendations: "
    sending = {"warmth":heat, "wetness":rain, "clothing":{}}
    for i in range(len(Temp_item[warmth])):
        if (i !=(len(Temp_item[warmth])-1)):
            sending["clothing"]["warm1"] = Temp_item[warmth][i]
        else:
            sending["clothing"]["warm2"] = Temp_item[warmth][i]
    for j in range(len(Rain_item[wet])):
        if (j !=(len(Rain_item[wet])-1)):
            sending["clothing"]["prep1"] = Rain_item[wet][j]
        else:
            sending["clothing"]["prep2"] = Rain_item[wet][j]
    return jsonify(sending)
#     return heat

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jan/2021 02:42:07] "GET /weather HTTP/1.1" 200 -
